# OLG Model Integration Module

This module provides a simple interface to run OLG transition analysis
using the functions defined in olg_transition_solver.

In [1]:
import pickle
import os
from datetime import datetime
from olg.transition.setting import TransitionSetting
from olg.transition.main import (
    run_transition_analysis,
)

In [2]:
def save_results_to_pkl(results, output_dir="results", timestamp=False):
    """
    結果をpickleファイルとして保存

    Parameters
    ----------
    results : tuple
        (initial_result, final_result, K_path, opt_indexes, aprimes, value_functions)
    prefix : str
        ファイル名のプレフィックス
    timestamp : bool
        タイムスタンプをファイル名に含めるか
    """
    # 結果をアンパック
    initial_result, final_result, K_path, opt_indexes, aprimes, value_functions = (
        results
    )

    # タイムスタンプ付きのディレクトリを作成
    if timestamp:
        timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_dir = f"{output_dir}_{timestamp_str}"
    else:
        output_dir = f"{output_dir}"

    os.makedirs(output_dir, exist_ok=True)

    # 各結果をpklファイルとして保存
    components = {
        "initial_result": initial_result,
        "final_result": final_result,
        "K_path": K_path,
        "opt_indexes": opt_indexes,
        "aprimes": aprimes,
        "value_functions": value_functions,
    }

    saved_files = []
    for name, data in components.items():
        filename = f"{output_dir}/{name}.pkl"
        with open(filename, "wb") as f:
            pickle.dump(data, f)
        saved_files.append(filename)
        print(f"Saved: {filename}")

    # 全体も一つのファイルとして保存
    full_filename = f"{output_dir}/full.pkl"
    with open(full_filename, "wb") as f:
        pickle.dump(results, f)
    saved_files.append(full_filename)
    print(f"Saved: {full_filename}")

    return saved_files

In [14]:
def run_custom_transition_analysis():
    """カスタム設定で移行過程分析を実行"""

    # カスタム移行過程設定を作成
    tr_setting = TransitionSetting(
        NT=5,  # 移行期間
        TT=5,  # 政策変更期間
        psi_ini=0.5,  # 初期所得代替率
        psi_fin=0.25,  # 最終所得代替率
    )

    # 初期・最終定常状態用のSetting作成
    initial_setting, final_setting = tr_setting.create_ss_settings(
        Na=101,
        Naprime=2001,  # 資産グリッド数  # 政策関数用資産グリッド数
    )

    # 移行過程分析を実行
    K_path_guess = pickle.load(
        open("K_path.pkl", "rb")
    )
    results = run_transition_analysis(
        tr_setting, initial_setting, final_setting, K_path_guess=K_path_guess
    )

    return results

In [15]:
%%time
# メイン実行部分
results = run_custom_transition_analysis()

print("\n=== カスタム設定結果の保存 ===")
saved_files = save_results_to_pkl(results, output_dir="psi_50to25_by5T")

print("\n=== 分析完了 ===")
print("以下のpklファイルとして保存されました:")
print("\nカスタム設定:")
for f in saved_files:
    print(f"  {f}")

=== OLG移行過程分析 ===
移行期間: 5 期
初期定常状態設定: ψ = 0.500
最終定常状態設定: ψ = 0.250

=== 初期定常状態の計算 ===
Iteration 1: market_diff = 6.803420e-01, errm = 0.000000e+00
Iteration 2: market_diff = 1.284662e-01, errm = 0.000000e+00
Iteration 3: market_diff = 2.420232e-02, errm = 0.000000e+00
Iteration 4: market_diff = 3.463949e-03, errm = 1.110223e-16
Iteration 5: market_diff = 1.147234e-03, errm = 0.000000e+00
\n計算完了! 実行時間: 3.50秒
総イテレーション数: 6
最終市場差: 1.292885e-04
最終人口合計誤差: 0.000000e+00
収束した資本ストック K = 6.1675

=== 最終定常状態の計算 ===
Iteration 1: market_diff = 3.147847e+00, errm = 0.000000e+00
Iteration 2: market_diff = 7.246067e-01, errm = 0.000000e+00
Iteration 3: market_diff = 2.255980e-01, errm = 0.000000e+00
Iteration 4: market_diff = 7.502705e-02, errm = 0.000000e+00
Iteration 5: market_diff = 2.365436e-02, errm = 0.000000e+00
\n計算完了! 実行時間: 3.81秒
総イテレーション数: 9
最終市場差: 5.214538e-04
最終人口合計誤差: 0.000000e+00
収束した資本ストック K = 6.8422

=== 移行過程の初期設定 ===

=== 移行過程の反復計算 ===
Iteration 1
  期間 5/5 を計算中...
  期間 4/5 を計算中...
  期間